In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [80]:
import numpy as np
import pandas as pd
# update 순서
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train.csv')

## 
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_CryoSleep_0.csv')
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Agelow12.csv')

# 정확도 0.9427710843373494
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_HomePlanet_update.csv')

# 정확도 0.9571514785757392
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_CryoSleep_update.csv')

# 정확도 0.9734939759036144
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_VIP_update.csv')

# 정확도 0.7013221153846154
df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Destination_update.csv')



### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [81]:
df_train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_num         0
Cabin_port        0
Destination     178
Age             175
VIP             197
RoomService     158
FoodCourt       166
ShoppingMall    186
Spa             167
VRDeck          160
Name            198
Transported       0
dtype: int64

In [82]:
df_train = df
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

In [84]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def fillnullclassifier(column_name):

    global df_update

    df_column_name = df_train[df_train[column_name].isnull()]

    df_train01 = df_train.dropna(subset=[column_name])

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)
    model = xgb.XGBClassifier(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)
    acc = accuracy_score(y_test,pred)
    print(acc)
    
    df_column_name_null = df_column_name.drop(column_name,axis=1)

    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))

    df_column_name[column_name] = pred

    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnullclassifier('Destination')

0.7013221153846154


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_9744\1768376489.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [64]:
# df_update

In [86]:
df_update[df_update['Destination'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [87]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_Destination_update.csv', index=False)